In [1]:
import tensorflow as tf
import keras 
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras import regularizers
import pandas as pd
import numpy as np
import os
import datetime
from keras.callbacks import EarlyStopping, TensorBoard

In [2]:
filepath = r"C:\Users\User\Downloads\archive\heart.csv"
heart_disease = pd.read_csv(filepath)
heart_disease.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
5,58,0,0,100,248,0,0,122,0,1.0,1,0,2,1
6,58,1,0,114,318,0,2,140,0,4.4,0,3,1,0
7,55,1,0,160,289,0,0,145,1,0.8,1,1,3,0
8,46,1,0,120,249,0,0,144,0,0.8,2,0,3,0
9,54,1,0,122,286,0,0,116,1,3.2,1,2,2,0


In [3]:
heart_disease_features = heart_disease.drop('target',axis=1)
heart_disease_label = heart_disease['target']

In [4]:
numpy_features = heart_disease_features.to_numpy()
numpy_label = heart_disease_label.to_numpy()

In [5]:
numpy_label = numpy_label.reshape(1025,1)

In [6]:
print(f"The shape of features: {numpy_features.shape}")
print(f"The shape of label: {numpy_label.shape}")

The shape of features: (1025, 13)
The shape of label: (1025, 1)


In [7]:
fnn_model = keras.Sequential([
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

In [8]:
standardizer=StandardScaler()

In [9]:
training_loss = []
training_acc = []
val_loss = []
val_acc = []

loss_list = []
acc_list = []
precision_list = []
recall_list = []

In [10]:
adam = keras.optimizers.Adam(learning_rate=0.001)
bce = keras.losses.BinaryCrossentropy(from_logits=False)
accuracy = keras.metrics.BinaryAccuracy()
recall = keras.metrics.Recall()
precision = keras.metrics.Precision()

In [11]:
base_log_path = r"C:\Users\User\Downloads\heart_log"
log_path = os.path.join(base_log_path, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
es = EarlyStopping(monitor='val_loss',patience=5)
tb = TensorBoard(log_dir=log_path)

In [12]:
kFold = KFold(n_splits=5, shuffle=True, random_state=12345)

In [13]:
fold_no = 1
for train, test in kFold.split(numpy_features, numpy_label):
    numpy_features[train] = standardizer.fit_transform(numpy_features[train])  #do data normalization to remove outliers, biases etc.
    numpy_features[test] = standardizer.transform(numpy_features[test])
    fnn_model.compile(optimizer=adam, loss=bce, 
                  metrics=[accuracy, precision, recall])
    print('########################################################################')
    print(f'Training for fold number {fold_no}')
    history = fnn_model.fit(numpy_features[train], numpy_label[train],
                        validation_data=(numpy_features[test], numpy_label[test]),
                        batch_size=32, epochs=20, callbacks=[es,tb])
    training_loss.append(history.history['loss'])
    training_acc.append(history.history['binary_accuracy'])
    val_loss.append(history.history['val_loss'])
    val_acc.append(history.history['val_binary_accuracy'])
    scores = fnn_model.evaluate(numpy_features[test], numpy_label[test])
    print(f'Score for fold {fold_no}: ')
    loss_list.append(scores[0])
    acc_list.append(scores[1])
    precision_list.append(scores[2])
    recall_list.append(scores[3])
    for metric_name, score in zip(fnn_model.metrics_names, scores):
        print(f'{metric_name} : {score}')
        print()
    fold_no += 1
    keras.backend.clear_session()

########################################################################
Training for fold number 1
Epoch 1/20
26/26 [==============================] - 7s 77ms/step - loss: 0.6199 - binary_accuracy: 0.6634 - precision: 0.6548 - recall: 0.7639 - val_loss: 0.5092 - val_binary_accuracy: 0.8000 - val_precision: 0.7265 - val_recall: 0.9043
Epoch 2/20
26/26 [==============================] - 1s 26ms/step - loss: 0.4861 - binary_accuracy: 0.7902 - precision: 0.7675 - recall: 0.8634 - val_loss: 0.3820 - val_binary_accuracy: 0.8439 - val_precision: 0.7925 - val_recall: 0.8936
Epoch 3/20
26/26 [==============================] - 1s 25ms/step - loss: 0.4274 - binary_accuracy: 0.8171 - precision: 0.8079 - recall: 0.8565 - val_loss: 0.3470 - val_binary_accuracy: 0.8390 - val_precision: 0.7748 - val_recall: 0.9149
Epoch 4/20
26/26 [==============================] - 1s 29ms/step - loss: 0.3798 - binary_accuracy: 0.8451 - precision: 0.8238 - recall: 0.8981 - val_loss: 0.3333 - val_binary_accuracy: 0.85

26/26 [==============================] - 1s 24ms/step - loss: 0.1141 - binary_accuracy: 0.9622 - precision: 0.9667 - recall: 0.9599 - val_loss: 0.0846 - val_binary_accuracy: 0.9561 - val_precision: 0.9697 - val_recall: 0.9412
Epoch 14/20
26/26 [==============================] - 1s 24ms/step - loss: 0.1167 - binary_accuracy: 0.9585 - precision: 0.9514 - recall: 0.9693 - val_loss: 0.0722 - val_binary_accuracy: 0.9659 - val_precision: 0.9703 - val_recall: 0.9608
Epoch 15/20
26/26 [==============================] - 1s 25ms/step - loss: 0.1174 - binary_accuracy: 0.9585 - precision: 0.9514 - recall: 0.9693 - val_loss: 0.0893 - val_binary_accuracy: 0.9561 - val_precision: 0.9697 - val_recall: 0.9412
Epoch 16/20
26/26 [==============================] - 1s 24ms/step - loss: 0.0976 - binary_accuracy: 0.9659 - precision: 0.9583 - recall: 0.9764 - val_loss: 0.0700 - val_binary_accuracy: 0.9659 - val_precision: 0.9703 - val_recall: 0.9608
Epoch 17/20
26/26 [==============================] - 1s 24ms

26/26 [==============================] - 1s 25ms/step - loss: 0.0400 - binary_accuracy: 0.9890 - precision: 0.9928 - recall: 0.9856 - val_loss: 0.0090 - val_binary_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 10/20
26/26 [==============================] - 1s 23ms/step - loss: 0.0361 - binary_accuracy: 0.9878 - precision: 0.9951 - recall: 0.9808 - val_loss: 0.0075 - val_binary_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 11/20
26/26 [==============================] - 1s 26ms/step - loss: 0.0313 - binary_accuracy: 0.9915 - precision: 0.9928 - recall: 0.9904 - val_loss: 0.0105 - val_binary_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 12/20
26/26 [==============================] - 1s 24ms/step - loss: 0.0459 - binary_accuracy: 0.9890 - precision: 0.9880 - recall: 0.9904 - val_loss: 0.0075 - val_binary_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 13/20
26/26 [==============================] - 1s 23ms

In [16]:
print('Average Loss :',np.mean(loss_list))
print('Average Accuracy :', np.mean(acc_list))
print('Average Precision :', np.mean(precision_list))
print('Average Recall :', np.mean(recall_list))

Average Loss : 0.06481270911172032
Average Accuracy : 0.978536593914032
Average Precision : 0.9761770009994507
Average Recall : 0.9798997163772583
